In [55]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv('../output/results.csv', engine='pyarrow')
df = df[df['tech'] != 'm1'] # This is merging files on the laptop
df['function'] = df['function'].str.replace('_',' ')
df.head()

,time,function,tech,merged,filename,step,file_mb,row_count,sent_mb,recv_mb,sent_mb_1s,recv_mb_1s,error,data_dir,file_count,datetime,id
7,38.518160,dvc upload new,dvc,False,fhvhv_tripdata_2020-01.parquet,0,506.664212,20569368,530.850586,4.137695,0.013672,0.000977,None,data,37,02/08/2023 01:49:49,1765052503727623331
8,84.826982,lfs s3 upload new,lfs-s3,False,fhvhv_tripdata_2020-01.parquet,0,506.664212,20569368,985.378906,646.531250,0.308594,0.004883,None,data,37,02/08/2023 01:49:49,1765052503727623331
9,23.489945,lfs github upload new,lfs-github,False,fhvhv_tripdata_2020-01.parquet,0,506.664212,20569368,536.644531,4.572266,0.284180,0.004883,None,data,37,02/08/2023 01:49:49,1765052503727623331
10,138.476349,xethub upload new,xethub,False,fhvhv_tripdata_2020-01.parquet,0,506.664212,20569368,582.905273,6.609375,0.284180,0.004883,None,data,37,02/08/2023 01:49:49,1765052503727623331
11,5.649378,dvc upload merged,dvc,True,fhvhv_tripdata_2020-01.parquet,0,506.664212,20569368,1.806641,0.109375,0.365234,0.003906,None,data,37,02/08/2023 01:49:49,1765052503727623331


In [56]:
df['error'].value_counts()

Series([], Name: count, dtype: int64)

In [57]:
for step in set(df['step']):
    print(f"Step {step} took {df[df['step']==step]['time'].sum()/60:.2f} minutes")
print(f"total time is: {df['time'].sum()/(60*60):.2f} hours")

Step 0 took 6.03 minutes
Step 1 took 38.55 minutes
Step 2 took 29.62 minutes
Step 3 took 17.21 minutes
Step 4 took 30.96 minutes
Step 5 took 35.59 minutes
Step 6 took 29.79 minutes
Step 7 took 24.51 minutes
Step 8 took 37.87 minutes
Step 9 took 41.24 minutes
Step 10 took 47.83 minutes
Step 11 took 49.38 minutes
Step 12 took 51.98 minutes
Step 13 took 45.75 minutes
Step 14 took 74.83 minutes
Step 15 took 69.52 minutes
total time is: 10.51 hours


In [65]:
groups = df.groupby('function')

In [66]:
import plotly.express as px

grouped = groups.sum()
fig = px.pie(grouped, values='time', names=grouped.index)

# Customize the plot as needed
fig.update_traces(textposition='inside', textinfo='percent+label')

# Show the plot
fig.show()

In [67]:
fig = go.Figure()
for name, group in groups:
    fig.add_trace(go.Scatter(x=group['step'], y=group['time'], mode='lines', name=str(name)))
fig.update_layout(
    xaxis_title='Step',
    yaxis_title='Time'
)
fig.show()

In [68]:
grouped = df.groupby('function')['time'].cumsum()

# Add the cumulative sum as a new column to the DataFrame
df['cumulative_time'] = grouped

# Create a Plotly line chart with the cumulative sum of time per tech
fig = px.line(df, x='step', y='cumulative_time', color='function')

# Customize the plot as needed
fig.update_layout(title='Cumulative Lines of Time per Steps',
                  xaxis_title='Steps',
                  yaxis_title='Cumulative Time')

# Show the plot
fig.show()

In [69]:
grouped = df.groupby('tech')['time'].cumsum()

# Add the cumulative sum as a new column to the DataFrame
df['cumulative_time'] = grouped

# Create a Plotly line chart with the cumulative sum of time per tech
fig = px.line(df, x='step', y='cumulative_time', color='tech')

# Customize the plot as needed
fig.update_layout(title='Cumulative Lines of Time per Steps',
                  xaxis_title='Steps',
                  yaxis_title='Cumulative Time')

# Show the plot
fig.show()

In [71]:
import plotly.express as px

grouped = df[df['step']==15].groupby('function').sum()
fig = px.pie(grouped, values='time', names=grouped.index)

# Customize the plot as needed
fig.update_traces(textposition='inside', textinfo='percent+label')

# Show the plot
fig.show()